# CIwA_Demo_Tutorial

This notebook demonstrates initializing, configuring, and running a CIwA process. It also covers the meaning and use of various components such as Process, Session, Topic, Submission, VotingManager, VotingMethod, Participant, and VotingResults.

## Install necessary packages if not already installed

In [ ]:
!pip install matplotlib networkx pyyaml jsonschema ipywidgets nest_asyncio


## Set up the Python path to include the parent directory of the ciwa module

In [ ]:
import sys
import os

# Add the parent directory of the 'ciwa' module to the Python path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Import necessary modules

In [ ]:
import asyncio
from ciwa.config.config_manager import ConfigManager
from ciwa.models.process import ProcessFactory
from ciwa.models.session import SessionFactory
from ciwa.models.topic import TopicFactory
from ciwa.models.participants.participant_factory import ParticipantFactory
from ciwa.utils.notebook_utils import visualize_session, display_results

## Initialize ConfigManager from a configuration dictionary

In [ ]:
config_dict = {
    "process": {
        "name": "CIwA Demo Process",
        "description": "A demo process for CIwA",
        "default_session_settings": {
            "max_subs_per_topic": 5, # number of submissions a participant can submit per topic
            "max_concurrent": 100, # limit of concurrent threads (for saving compute if needed)
            "default_topic_settings": {
                "voting_method": {
                    "type": "RankingCompare"
                }
            },
            "participants": [
                {
                    "type": "LLMAgentParticipant", # this is a demo participant that generates dummy code
                    "model": "demo"
                }
            ]
        },
        "sessions": [
            {
                "name": "Demo Session",
                "description": "A demo session for CIwA",
                "topics": [
                    {
                        "title": "What is a good topic to test this on?",
                        "description": "Give a submission for what would be a good topic to test this on."
                    }
                ]
            }
        ]
    }
}

config_manager = ConfigManager(config=config_dict)

## Create a Process

In [ ]:
process_config = config_manager.get_config("process")
process = ProcessFactory.create_process(**process_config)

## Add a new session to the process

In [ ]:
new_session_config = {
    "name": "New Demo Session",
    "description": "Another demo session for CIwA using LLMs",
    "topics": [
        {
            "title": "What is a challenge that advanced AI technology poses"\
                     "to the collective benefit of humanity?",
            "description": "Submit one sentence describing a specific challenge"\
                     "AI poses to the collective benefit of humanity."
        }
    ],
    "participants": [
        {
            "type": "ConversableAgentParticipant",
            "model": "gpt-3.5-turbo"
        }
    ]
}

process.add_session(new_session_config)

## Add more participants to a session

In [ ]:
# Set a number of participants
num_participants = 5

# Define the participant configuration
participant_config = {
    "type": "ConversableAgentParticipant",
    "model": "gpt-3.5-turbo",
    "temperature": 0.9, # higher temperature (0-1) increases randomness/diversity of output
    "cache_seed": None, # set to None to avoid caching LLM replies
}

# Create a list of participant configurations
list_of_participant_configs = [participant_config for _ in range(num_participants)]

# add participants to a pending session
process.pending_sessions[1].add_participants(list_of_participant_configs)


## Visualize the process and sessions

In [ ]:
print("Process and Sessions Initialized")
print(f"Process: {process.name}")
for session in process.pending_sessions:
    print(f"Session: {session.name}, Description: {session.description}")
    for participant in session.participants:
        print(f"Participant: {participant.type}, Model: {participant.model}")

## Run a session and gather submissions

In [ ]:
# Use the nest_asyncio package to allow nested use of asyncio.run() in a Notebook
import nest_asyncio
nest_asyncio.apply()

async def run_session():
    await process.run_next_session()

await run_session()

## Visualize session results

In [ ]:
session = process.completed_sessions[-1]
visualize_session(session)
display_results(session)

## Run the next session and visualize results

In [ ]:
await run_session()

In [ ]:
session = process.completed_sessions[-1]
visualize_session(session)
display_results(session)